In [1]:
fp = f"/Users/gugujixiao/workspace/project/HWMemory/Code/HW-Memory/example/old_example/func.asm"

In [2]:
from sample.cfg import proc_identify
from sample.isa import Instruction, AddrMode
from typing import Tuple
from sample.read_asm import AsmFileReader, StatementType

reader =AsmFileReader(fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

#for i in temp_stat:
#    booltemp = i.is_ls
#    if booltemp:
#        if i.ls_addr_mode == AddrMode.ImmeAft:
#            print(i.ls_reg_traget)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
        print()

_init 0000000000400498
.plt 00000000004004b0
__libc_start_main@plt 00000000004004d0
__gmon_start__@plt 00000000004004e0
abort@plt 00000000004004f0
_start 0000000000400500
__wrap_main 0000000000400534
_dl_relocate_static_pie 0000000000400540
call_weak_fn 0000000000400544
deregister_tm_clones 0000000000400560
register_tm_clones 0000000000400590
__do_global_dtors_aux 00000000004005d0
frame_dummy 0000000000400600
compare 0000000000400604
main 0000000000400634
_fini 0000000000400658


In [3]:
from graphviz import Digraph
from sample.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
from sample.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [6]:
from sample.cfg import TCfg
from sample.rw_condition import hw_CFG_Node

tcfg = TCfg(call_graph)
tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

hw_cfg_node_list = list()

for node in tcfg_nodes:
    #print(node.name, node.base_proc.name)
    #print(node.name, node.base_proc.name, node.inst_range)\
    #print("入边")
    #for i in node.to_edge:
    #    print(i.src.name)
    #print("出边")
    #for i in node.from_edge:
    #    print(i.dst.name)
    
    #instr = node.instructions
    #for ins in instr:
    #    booltemp = ins.is_ls
    #    if booltemp:
    #        print(ins.tokens)
    #print()
    #print()
    temp = hw_CFG_Node(node)
    hw_cfg_node_list.append(temp)

for i in hw_cfg_node_list:
    print(i.node_self.name)
    print("来的点")
    for j in i.node_from:
        print(j.name)
    print("出去的点")
    for k in i.node_to:
        print(k.name)
    print("指令")
    for a in i.instr:
        print(a.tokens)
    print()

#for edge in tcfg_edges:
#    print(edge.src.name, edge.dst.name, edge.kind.name)

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

n0
来的点
出去的点
n2
指令
('400634', 'a9be7bfd', 'stp', None, 'x29, x30, [sp, #-32]!')
('400638', '910003fd', 'mov', None, 'x29, sp')
('40063c', '528001e0', 'mov', None, 'w0, #0xf                        ')
('400640', 'b9001fe0', 'mov', None, 'x33, sp')
('400644', 'b9401fe0', 'ldr', None, 'w0, [x12], #12')
('400648', '97ffffef', 'bl', None, '400604 <compare>')

n1
来的点
n5
出去的点
指令
('40064c', '52800000', 'mov', None, 'w0, #0x0                        ')
('400650', 'a8c27bfd', 'ldp', None, 'x29, x30, [sp], #32')
('400654', 'd65f03c0', 'ret', None, '')

n2
来的点
n0
出去的点
n4
n3
指令
('400604', 'd10083ff', 'sub', None, 'sp, sp, #0x20')
('400608', 'b9000fe0', 'str', None, 'w0, [sp, #12]')
('40060c', '52800140', 'mov', None, 'w0, #0xa                        ')
('400610', 'b9001fe0', 'adrp', None, 'x33, 41f000 <__FRAME_END__+0x1e868>')
('400614', 'b9400fe0', 'ldr', None, 'w0, [x33, #0x12]!')
('400618', '7100281f', 'cmp', None, 'w0, #0xa')
('40061c', '5400006d', 'b', 'le', '400628 <compare+0x24>')

n3
来的点
n2
出去

'output/tcfg.svg'